In [1]:
import numpy as np
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.coordinates import EarthLocation, get_body
from astropy import units as u
from sgl_lib import sgl_position, sgl_position_scatter

Define object of interest and planned observation time from Earth

In [2]:
object_name = 'Tau Ceti'   # Tau Ceti
observation_time = 2020.2 * u.year # UTC time of planned observation from Earth

Use object name to pull the Gaia DR2 identifier from Vizier

In [3]:
result = str(Simbad.query_objectids(object_name))
position = result.find('Gaia DR2 ') + 9
gaia_ID = result[position:position+19]
print('Gaia ID:', gaia_ID)

Gaia ID: 2452378776434276992


With this Gaia ID we pull the catalog data of object position, parallax, and proper motion

In [4]:
result = Vizier().query_constraints(Source=gaia_ID, catalog="I/345/gaia2")[0][:][0]
RA     = result['RA_ICRS']   * u.degree      # Position in RA, epoch 2015.5
e_RA   = result['e_RA_ICRS'] * u.arcsec      # Uncertainty in the position in RA
DE     = result['DE_ICRS']   * u.degree      # Position in DE, epoch 2015.5
e_DE   = result['e_DE_ICRS'] * u.arcsec      # Uncertainty in the position in DE
Plx    = result['Plx']    / 1000 * u.arcsec  # Parallax (as)
e_Plx  = result['e_Plx']  / 1000 * u.arcsec  # Uncertainty in the parallax (as)

pmRA   = result['pmRA']   / 1000 * u.arcsec / u.year  # proper motion in RA (as/yr)
e_pmRA = result['e_pmRA'] / 1000 * u.arcsec / u.year  # proper motion in DE (as/yr)
pmDE   = result['pmDE']   / 1000 * u.arcsec / u.year  # Uncertainty in the proper motion in RA (as/yr)
e_pmDE = result['e_pmDE'] / 1000 * u.arcsec / u.year  # Uncertainty in the proper motion in DE (as/yr)

# Derived values
epoch = 2015.5 * u.year  # Gaia epoch J2015.5 for DR2
distance = Plx.to(u.parsec, equivalencies=u.parallax())
distance_uncertainty = - distance + (Plx - e_Plx).to(u.parsec, equivalencies=u.parallax())

print('RA', RA, '+-', e_RA)
print('DE', DE, '+-', e_DE)
print('Plx (as)', Plx, '+-', e_Plx)
print('Proper motion in RA (as/yr)', pmRA, '+-', e_pmRA)
print('Proper motion in DE (as/yr)', pmDE, '+-', e_pmDE)
print('Distance (pc)', distance, '+-', distance_uncertainty)

RA 26.00930287667 deg +- 0.4122 arcsec
DE -15.93379865094 deg +- 0.3505 arcsec
Plx (as) 0.27751620000000005 arcsec +- 0.000517300009727478 arcsec
Proper motion in RA (as/yr) -1.729726 arcsec / yr +- 0.001312999963760376 arcsec / yr
Proper motion in DE (as/yr) 0.8554930000000001 arcsec / yr +- 0.0007829999923706054 arcsec / yr
Distance (pc) 3.6033932433493967 pc +- 0.006729396253566833 pc


Tau Ceti position at observation epoch

In [19]:
time_gap = observation_time - epoch
print('Time gap', time_gap)
# Error in RA, DE
error_ra = e_RA+e_pmRA*time_gap
error_de = e_DE+e_pmDE*time_gap
print('Error in position at 2020.2 (RA, DEC)', RA+error_ra, RA+error_de)

pos_nominal = sgl_position(
    RA=RA,
    DE=DE,
    distance=distance,
    time=Time(observation_time.value, format='decimalyear'),
    z=1000*u.au,  # heliocentric probe position on focal line
    verbose=True
    )
print(pos_nominal)
print('distance', pos_nominal.distance.to(u.au))

pos_error = sgl_position(
    RA=RA+error_ra,
    DE=DE+error_de,
    distance=distance,
    time=Time(observation_time.value, format='decimalyear'),
    z=1000*u.au,  # heliocentric probe position on focal line
    )
print('error', (pos_error.ra - pos_nominal.ra))
print('error', (pos_error.dec - pos_nominal.dec))

pos_after58d = sgl_position(
    RA=RA,
    DE=DE,
    distance=distance,
    time=Time(observation_time.value + 5.8/365, format='decimalyear'),
    z=1000*u.au,  # heliocentric probe position on focal line
    )

print('Separation after light travel time', pos_nominal.separation(pos_after58d))


Time gap 4.7000000000000455 yr
Error in position at 2020.2 (RA, DEC) 26.009419090864395 deg 26.0094012600311 deg
ltt 0.01581250740982066 yr
sun  xyz 0.9710382735608333 AU -0.18896080062523246 AU -0.08191188756022487 AU
star xyz 3.114029044866842 pc 1.5194393849159522 pc -0.9892269352924127 pc
star xyz 642314.5975822574 AU 313406.87033144064 AU -204042.70214089978 AU
length_of_vector 3.6033894522383605 pc
sgl -863.2220716901896 AU -421.8587029041097 AU 274.4447667532638 AU
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (206.0448711, 15.94172312, 0.00484435)>
distance 999.2182145393938 AU
error 0d00m00.4187s
error -0d00m00.3545s
Separation after light travel time 0d00m18.8494s


In [21]:
# Light travel time to Tau Ceti
ltt_tau_ceti = distance.to(u.lightyear).value * u.year
print(ltt_tau_ceti)

pos_transmitter = sgl_position(
    RA=RA-pmRA*ltt_tau_ceti,
    DE=DE-pmDE*ltt_tau_ceti,
    distance=distance,
    time=Time(observation_time.value, format='decimalyear'),
    z=1000*u.au,  # heliocentric probe position on focal line
    )
print(pos_transmitter)
print('3D distance', pos_nominal.separation_3d(pos_transmitter).to(u.au))
print('3D distance', pos_nominal.separation_3d(pos_transmitter).to(u.km))

11.752696877306187 yr
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (206.05052317, 15.94451725, 0.00484435)>
3D distance 0.10657148259297843 AU
3D distance 15942866.873251686 km
